# Import libraries and set up database connection

In [2]:
from dotenv import load_dotenv
from os import getenv
import pandas as pd
from IPython.display import display, HTML
import yfinance as yf
import numpy as np
import holidays
from functools import reduce

load_dotenv()
DB_HOST = getenv('DB_HOST')
DB_USER = getenv('DB_USER')
DB_PASSWORD = getenv('DB_PASSWORD')

from sqlalchemy import create_engine
cnx = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:3306/ats_db')

# Fetch stock symbols from database

In [3]:
companies = pd.read_sql_query(f'''SELECT companyName, symbol FROM companies ORDER BY companyName;''', con=cnx)
display(HTML(companies.to_html()))

,companyName,symbol
0,3M Company,MMM
1,A. O. Smith Corporation,AOS
2,Abbott Laboratories,ABT
3,AbbVie Inc.,ABBV
4,Accenture plc,ACN
5,Adobe Inc.,ADBE
6,"Advanced Micro Devices, Inc.",AMD
7,Aflac Incorporated,AFL
8,"Agilent Technologies, Inc.",A
9,"Air Products and Chemicals, Inc.",APD


# Source demographics data

In [4]:
# Extract symbols from companies information
tickers = companies.symbol.tolist()

def extract_demographics(x, to_extract):
    # Given a dictionary and a list of keys to extract, will return as a dictionary
    # if the value does not exist on the yahoo finance site, np.nan will fill the value
    result_dict = {key: x.get(key, np.nan) for key in to_extract}
    return result_dict
    
# Ratios to extract from api
ratios_to_extract = ['symbol', 'sector', 'trailingPE' , 'priceToBook', 'debtToEquity', 'freeCashflow',
                     'pegRatio' , 'returnOnEquity']

demographic_df = pd.DataFrame()

for i in range(len(tickers)):
    temp_ticker = yf.Ticker(tickers[i]).info
    temp_row = extract_demographics(temp_ticker, ratios_to_extract)
    temp_df  = pd.DataFrame([temp_row])
    demographic_df = pd.concat([demographic_df, temp_df],ignore_index=True)

display(demographic_df)

,symbol,sector,trailingPE,priceToBook,debtToEquity,freeCashflow,pegRatio,returnOnEquity
0,MMM,Industrials,NaN,11.731863,451.166,8.903375e+09,-2.71,-0.69286
1,AOS,Industrials,21.303896,6.392829,7.764,4.549750e+08,2.03,0.31443
2,ABT,Healthcare,32.601246,4.690932,37.664,4.607125e+09,2.81,0.14762
3,ABBV,Healthcare,50.444780,37.263508,924.059,2.361150e+10,2.45,0.56244
4,ACN,Technology,27.790102,6.847728,16.078,8.045156e+09,3.57,0.26000
...,...,...,...,...,...,...,...,...
498,XYL,Industrials,47.522650,3.244214,23.268,5.701250e+08,2.21,0.09614
499,YUM,Consumer Cyclical,23.554174,NaN,NaN,1.170750e+09,2.09,NaN
500,ZBRA,Technology,60.669960,4.979239,72.957,1.483750e+08,1.35,0.08614
501,ZBH,Healthcare,23.509760,1.769614,47.626,1.070800e+09,1.94,0.07795


# Extract historical stock values

In [15]:
stocks = pd.read_sql_query(f'''SELECT c.symbol as stock_ID, hv.volume as Volume, hv.close as Close, hv.date as DATETIME
    FROM historical_stock_values hv JOIN companies c ON hv.company_id=c.id;''', con=cnx)

# covert DATETIME column values to datetime format
stocks['DATETIME'] = pd.to_datetime(stocks['DATETIME'], format ='%Y-%m-%d %H:%M:%S')
display(stocks)

,stock_ID,Volume,Close,DATETIME
0,A,1583790,72.14,2019-06-26
1,A,1480248,73.19,2019-06-27
2,A,3159736,74.67,2019-06-28
3,A,2756169,75.78,2019-07-01
4,A,1876980,75.61,2019-07-02
...,...,...,...,...
627082,ZTS,2185528,170.68,2024-06-21
627083,ZTS,1069707,172.53,2024-06-24
627084,ZTS,1273811,168.30,2024-06-25
627085,ZTS,1296029,171.44,2024-06-26


# Extract index, commodity, and bond values

In [16]:
# Dictionary to store commodites data
commodities = {}

In [17]:
# Extract index values

# Available indexes:
# S&P 500, NASDAQ Composite, Dow Jones Industrial Average, Russell 3000, FTSE 100
indexNames = [
'S&P 500',
'NASDAQ Composite',
'Dow Jones Industrial Average'
]
for indx in indexNames:
    commodities[indx] = pd.read_sql_query(f'''SELECT hv.date as DATETIME, hv.close as Close FROM historical_index_values hv
    JOIN indexes ind ON hv.index_id = ind.id WHERE ind.indexName = '{indx}';''', con=cnx)
display(commodities['S&P 500'])

,DATETIME,Close
0,2019-06-26,2913.78
1,2019-06-27,2924.92
2,2019-06-28,2941.76
3,2019-07-01,2964.33
4,2019-07-02,2973.01
...,...,...
1255,2024-06-21,5462.73
1256,2024-06-24,5465.25
1257,2024-06-25,5463.03
1258,2024-06-26,5467.29


In [18]:
# Extract commodity values

# available commodities:
# Crude Oil, Gold Futures, Copper, Silver Futures, Natural Gas
commodityNames = [
'Crude Oil',
'Gold Futures'
]
for comm in commodityNames:
    commodities[comm] = pd.read_sql_query(f'''SELECT hv.date as DATETIME, hv.close as Close FROM historical_commodity_values hv
    JOIN commodities c ON hv.commodity_id = c.id WHERE c.commodityName = '{comm}';''', con=cnx)
display(commodities['Crude Oil'])

,DATETIME,Close
0,2019-06-26,59.38
1,2019-06-27,59.43
2,2019-06-28,58.47
3,2019-07-01,59.09
4,2019-07-02,56.25
...,...,...
1259,2024-06-23,80.67
1260,2024-06-24,81.64
1261,2024-06-25,80.72
1262,2024-06-26,80.90


In [19]:
# Extract bond values

# available maturities:
# 1_month, 2_month, 3_month, 6_month, 1_year, 2_year, 3_year, 5_year, 7_year, 10_year, 20_year, 30_year
maturities = ['2_year', '5_year', '10_year']
bonds = pd.read_sql_query(f'''SELECT date as DATETIME, {', '.join(maturities)} FROM bond_values bv
JOIN bonds b ON bv.bond_id = b.id;''', con=cnx)

# add bonds to commodities dictionary
for mt in maturities:
    commodities[mt] = bonds.filter(['DATETIME', f'{mt}']).rename(columns = {f'{mt}': 'Close'})
    # display(commodities[mt])
# display(bonds)

# Merging commodities

In [20]:

for comm in commodities.keys():
    # change data type 
    commodities[comm]['DATETIME'] = pd.to_datetime(commodities[comm]['DATETIME'], format ='%Y-%m-%d %H:%M:%S')
    #move time period back by 15 minutes
    # commodities[comm]['DATETIME'] = commodities[comm]['DATETIME'] - pd.Timedelta(minutes=15)

# convert dictionary to array and rename columns
commoditiesReduced = []
for comm in commodities.keys():
    commoditiesReduced.append(commodities[comm].rename(columns = {'Close': f'{comm.replace(" ", "")}_PP'}))


In [21]:
display(commoditiesReduced[5])

,DATETIME,2_year_PP
0,2019-06-10,1.90
1,2019-06-11,1.93
2,2019-06-12,1.88
3,2019-06-13,1.83
4,2019-06-14,1.84
...,...,...
1260,2024-06-21,4.70
1261,2024-06-24,4.71
1262,2024-06-25,4.65
1263,2024-06-26,4.71


In [22]:
#use reduce function to merge list of dataframes into singular dataframe on DATETIME
mergedCommodities = reduce(lambda df1,df2: pd.merge(df1,df2,on='DATETIME'), commoditiesReduced)
display(mergedCommodities)

,DATETIME,S&P500_PP,NASDAQComposite_PP,DowJonesIndustrialAverage_PP,CrudeOil_PP,GoldFutures_PP,2_year_PP,5_year_PP,10_year_PP
0,2019-06-26,2913.78,7909.97,26536.82,59.38,1415.4,1.77,1.80,2.05
1,2019-06-27,2924.92,7967.76,26526.58,59.43,1412.0,1.74,1.76,2.01
2,2019-06-28,2941.76,8006.24,26599.96,58.47,1413.7,1.75,1.76,2.00
3,2019-07-01,2964.33,8091.16,26717.43,59.09,1389.3,1.78,1.79,2.03
4,2019-07-02,2973.01,8109.09,26786.68,56.25,1408.0,1.77,1.75,1.98
...,...,...,...,...,...,...,...,...,...
1246,2024-06-21,5462.73,17682.90,39142.77,80.58,2374.2,4.70,4.26,4.25
1247,2024-06-24,5465.25,17572.11,39418.08,81.64,2337.7,4.71,4.27,4.25
1248,2024-06-25,5463.03,17689.95,39125.53,80.72,2331.3,4.65,4.25,4.23
1249,2024-06-26,5467.29,17770.70,39163.65,80.90,2311.5,4.71,4.32,4.32


# Merging stock values and demographics

In [23]:
# Renaming demographics columns
demographic_df.rename(columns={
    'symbol': 'stock_ID',
    'sector':'Sector',
    'trailingPE' : 'P/E Ratio ttm',
    'priceToBook' : 'P/B Ratio mrq',
    'debtToEquity' : 'D/E Ratio mrq',
    'freeCashflow' : 'Free Cash Flow ttm',
    'pegRation' : 'Peg Ratio 5 year expected',
    'returnOnEquity' : 'ROE ttm'
}, inplace=True)

# Merging stocks with demographics
mergedStocks = pd.merge(stocks, demographic_df, on='stock_ID')
mergedStocks


,stock_ID,Volume,Close,DATETIME,Sector,P/E Ratio ttm,P/B Ratio mrq,D/E Ratio mrq,Free Cash Flow ttm,pegRatio,ROE ttm
0,A,1583790,72.14,2019-06-26,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709
1,A,1480248,73.19,2019-06-27,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709
2,A,3159736,74.67,2019-06-28,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709
3,A,2756169,75.78,2019-07-01,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709
4,A,1876980,75.61,2019-07-02,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709
...,...,...,...,...,...,...,...,...,...,...,...
627082,ZTS,2185528,170.68,2024-06-21,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047
627083,ZTS,1069707,172.53,2024-06-24,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047
627084,ZTS,1273811,168.30,2024-06-25,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047
627085,ZTS,1296029,171.44,2024-06-26,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047


# Merging commodities with stocks

In [24]:
df = pd.merge(mergedStocks, mergedCommodities, on='DATETIME')
display(df)

,stock_ID,Volume,Close,DATETIME,Sector,P/E Ratio ttm,P/B Ratio mrq,D/E Ratio mrq,Free Cash Flow ttm,pegRatio,ROE ttm,S&P500_PP,NASDAQComposite_PP,DowJonesIndustrialAverage_PP,CrudeOil_PP,GoldFutures_PP,2_year_PP,5_year_PP,10_year_PP
0,A,1583790,72.14,2019-06-26,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,2913.78,7909.97,26536.82,59.38,1415.4,1.77,1.80,2.05
1,A,1480248,73.19,2019-06-27,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,2924.92,7967.76,26526.58,59.43,1412.0,1.74,1.76,2.01
2,A,3159736,74.67,2019-06-28,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,2941.76,8006.24,26599.96,58.47,1413.7,1.75,1.76,2.00
3,A,2756169,75.78,2019-07-01,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,2964.33,8091.16,26717.43,59.09,1389.3,1.78,1.79,2.03
4,A,1876980,75.61,2019-07-02,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,2973.01,8109.09,26786.68,56.25,1408.0,1.77,1.75,1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622611,ZTS,2185528,170.68,2024-06-21,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,5462.73,17682.90,39142.77,80.58,2374.2,4.70,4.26,4.25
622612,ZTS,1069707,172.53,2024-06-24,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,5465.25,17572.11,39418.08,81.64,2337.7,4.71,4.27,4.25
622613,ZTS,1273811,168.30,2024-06-25,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,5463.03,17689.95,39125.53,80.72,2331.3,4.65,4.25,4.23
622614,ZTS,1296029,171.44,2024-06-26,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,5467.29,17770.70,39163.65,80.90,2311.5,4.71,4.32,4.32


In [25]:
df['Sector'].value_counts()

Sector
Technology                95076
Industrials               89896
Financial Services        82566
Healthcare                79261
Consumer Cyclical         70942
Consumer Defensive        45324
Real Estate               38781
Utilities                 38204
Basic Materials           27522
Energy                    27522
Communication Services    27522
Name: count, dtype: int64

# Creating Additional flags

In [26]:
#create columns for each categorical datetime variable of interest
df['DATE']        = df.DATETIME.dt.date
df['MONTH']       = df.DATETIME.dt.month
df['DAY']         = df.DATETIME.dt.day
df['HOUR']        = df.DATETIME.dt.hour
df['MINUTE']      = df.DATETIME.dt.minute
df['WEEK_DAY']    = df.DATETIME.dt.dayofweek

In [27]:
# Create holiday flag
holiday_days = []
for holiday in holidays.US(state = 'NY', years=[2019, 2020, 2021, 2022, 2023, 2024]).items(): holiday_days.append(str(holiday[0]))
df['HOLIDAY']=[1 if str(value) in holiday_days else 0 for value in df['DATE']]

In [28]:
# Create pre - post holiday flag
df['POSTHOLIDAY_MORNING'], df['PREHOLIDAY_AFTERNON'] = 0, 0

id = df.loc[(df.HOLIDAY == 1),:].index
POSTHOLIDAY = pd.to_datetime((df.loc[id,'DATE'] + pd.DateOffset(days=1)).unique())
PREHOLIDAY  = pd.to_datetime((df.loc[id,'DATE'] - pd.DateOffset(days=1)).unique())

for i in range(len(PREHOLIDAY)) : df.loc[(df.DATE == PREHOLIDAY[i]) & (df.HOUR > 12)   , 'PREHOLIDAY_AFTERNON'] =1
for i in range(len(POSTHOLIDAY)): df.loc[ (df.DATE == POSTHOLIDAY[i]) & (df.HOUR <= 12), 'POSTHOLIDAY_MORNING'] =1

In [29]:
# Create monday morning flag
df['MONDAY_MORNING']=0
df.loc[(df.WEEK_DAY == 0) & (df.HOUR <= 12), 'MONDAY_MORNING'] =1

In [30]:
# Create friday afternoon flag
df['FRIDAY_AFTERNOON']=0
df.loc[(df.WEEK_DAY == 4) & (df.HOUR > 12), 'FRIDAY_AFTERNOON'] =1

In [31]:
# Delete holiday days and date
df.drop(df.index[df['HOLIDAY']==1], inplace=True)
df.drop(columns=['HOLIDAY'], inplace=True)
df.drop(columns=['DATE'], inplace=True)

In [32]:
df

,stock_ID,Volume,Close,DATETIME,Sector,P/E Ratio ttm,P/B Ratio mrq,D/E Ratio mrq,Free Cash Flow ttm,pegRatio,...,10_year_PP,MONTH,DAY,HOUR,MINUTE,WEEK_DAY,POSTHOLIDAY_MORNING,PREHOLIDAY_AFTERNON,MONDAY_MORNING,FRIDAY_AFTERNOON
0,A,1583790,72.14,2019-06-26,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,...,2.05,6,26,0,0,2,0,0,0,0
1,A,1480248,73.19,2019-06-27,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,...,2.01,6,27,0,0,3,0,0,0,0
2,A,3159736,74.67,2019-06-28,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,...,2.00,6,28,0,0,4,0,0,0,0
3,A,2756169,75.78,2019-07-01,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,...,2.03,7,1,0,0,0,0,0,1,0
4,A,1876980,75.61,2019-07-02,Healthcare,30.995262,6.137675,43.869,9.242500e+08,5.20,...,1.98,7,2,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622611,ZTS,2185528,170.68,2024-06-21,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,...,4.25,6,21,0,0,4,0,0,0,0
622612,ZTS,1069707,172.53,2024-06-24,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,...,4.25,6,24,0,0,0,0,0,1,0
622613,ZTS,1273811,168.30,2024-06-25,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,...,4.23,6,25,0,0,1,0,0,0,0
622614,ZTS,1296029,171.44,2024-06-26,Healthcare,33.934616,15.941820,134.600,1.485375e+09,3.11,...,4.32,6,26,0,0,2,0,0,0,0


# One Hot Encoding

In [33]:
def one_hot(og_df, feature_to_encode):
    "Function that takes a dataframe, and a feature to one-hot encode and returns the dataframe with that feature encoded"
    dummies = pd.get_dummies(og_df[feature_to_encode], prefix = feature_to_encode, prefix_sep = "_", dtype=int)
    df = pd.concat([og_df, dummies], axis=1)
    df = df.drop([feature_to_encode], axis=1)
    return(df)

In [34]:
to_encode = ['Sector', 'WEEK_DAY', 'MINUTE', 'HOUR', 'MONTH', 'DAY']

In [35]:
#encode the time features of interest
encoded_df = df
for encode in to_encode:
    encoded_df = one_hot(encoded_df, encode)

In [36]:
encoded_df.rename(columns={"Volume":"Volume_PP"}, inplace=True)

In [37]:
encoded_df

,stock_ID,Volume_PP,Close,DATETIME,P/E Ratio ttm,P/B Ratio mrq,D/E Ratio mrq,Free Cash Flow ttm,pegRatio,ROE ttm,...,DAY_22,DAY_23,DAY_24,DAY_25,DAY_26,DAY_27,DAY_28,DAY_29,DAY_30,DAY_31
0,A,1583790,72.14,2019-06-26,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,...,0,0,0,0,1,0,0,0,0,0
1,A,1480248,73.19,2019-06-27,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,...,0,0,0,0,0,1,0,0,0,0
2,A,3159736,74.67,2019-06-28,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,...,0,0,0,0,0,0,1,0,0,0
3,A,2756169,75.78,2019-07-01,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,...,0,0,0,0,0,0,0,0,0,0
4,A,1876980,75.61,2019-07-02,30.995262,6.137675,43.869,9.242500e+08,5.20,0.20709,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622611,ZTS,2185528,170.68,2024-06-21,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,...,0,0,0,0,0,0,0,0,0,0
622612,ZTS,1069707,172.53,2024-06-24,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,...,0,0,1,0,0,0,0,0,0,0
622613,ZTS,1273811,168.30,2024-06-25,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,...,0,0,0,1,0,0,0,0,0,0
622614,ZTS,1296029,171.44,2024-06-26,33.934616,15.941820,134.600,1.485375e+09,3.11,0.50047,...,0,0,0,0,1,0,0,0,0,0


# Final Result

In [38]:
encoded_df['stock_ID'].value_counts()
encoded_df['DATETIME'].value_counts()

DATETIME
2024-06-27    503
2024-05-10    503
2024-05-08    503
2024-05-07    503
2024-05-06    503
             ... 
2019-12-18    494
2019-12-17    494
2019-12-16    494
2019-12-13    494
2019-06-26    494
Name: count, Length: 1235, dtype: int64

In [39]:
encoded_df.to_csv('df.csv', index=False)